Preprocessing From the previous assignment:

In [ ]:
from Data_Preprocessing import prepare_data
import pandas as pd
import numpy as np
import math
from numpy import nan
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
!rm *.csv
from google.colab import files
print("Please load the raw csv data")
uploadedData = files.upload()
filename = list(uploadedData)[0]
dataset = pd.read_csv(filename)
train_data, test_data= train_test_split(dataset, test_size=0.2, train_size=0.8, random_state=156)

dataset['risk'].replace({1: 1, 0: -1}, inplace=True)
train_data['risk'].replace({1: 1, 0: -1}, inplace=True)
test_data['risk'].replace({1: 1, 0: -1}, inplace=True)

dataset['spread'].replace({1: 1, 0: -1}, inplace=True)
train_data['spread'].replace({1: 1, 0: -1}, inplace=True)
test_data['spread'].replace({1: 1, 0: -1}, inplace=True)

dataset['covid'].replace({1: 1, 0: -1}, inplace=True)
train_data['covid'].replace({1: 1, 0: -1}, inplace=True)
test_data['covid'].replace({1: 1, 0: -1}, inplace=True)

# dataset, train_data = prepare_data(dataset, train_data)
test_data, train_data = prepare_data(test_data, train_data)


In [ ]:
# **Part 1: Basic model selection with k-Nearest Neighbors:**
from visualize_clf import *
# Visualization and basic analysis
# Q1:
temp_df = train_data[['PCR_08', 'PCR_10', 'risk']]
p = sns.jointplot(data = temp_df, x='PCR_08', y='PCR_10', hue = 'risk')
p.fig.suptitle("Marginal and joint distribution of PCR_08 and PCR_10 according to 'risk' label")

# Q2:
from sklearn.neighbors import KNeighborsClassifier
X = temp_df[['PCR_08', 'PCR_10']]
y = temp_df[['risk']]
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X, y)
visualize_clf(knn, X.to_numpy(), y.to_numpy(), 'Decision Reigons - k=1 neighbor', xlabel = 'PCR_08', ylabel = 'PCR_10')


# Q3:
from sklearn.model_selection import cross_validate
max = 0
k_opt = 0
k_range = list(range(1, 20, 1)) + list(range(20, 300, 10))
k_scores = []
k_train_scores = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_validate(knn, X, y, cv=8, return_train_score=True)
    mean = scores['test_score'].mean()
    mean_train = scores['train_score'].mean()
    if(mean > max):
      max = mean
      k_opt = k
    k_scores.append(mean)
    k_train_scores.append(mean_train)
plt.plot(k_range, k_scores, linewidth=2)
plt.plot(k_range, k_train_scores, linewidth=2)
plt.suptitle("Cross-Validated Accuracy vs. K value of knn")
plt.legend("Validation","Train")
plt.xlabel('Value of K for KNN')
plt.ylabel('Cross-Validated Accuracy')
plt.show()

print("the best k is: ", k_opt, "  and the mean is:", k_scores[k_opt])

# Q4:
X = temp_df[['PCR_08', 'PCR_10']]
y = temp_df[['risk']]
knn = KNeighborsClassifier(n_neighbors=18)
knn.fit(X, y)
visualize_clf(knn, X.to_numpy(), y.to_numpy(), 'Decision Reigons - k=18 neighbors', xlabel = 'PCR_08', ylabel = 'PCR_10')

print("with 18 neighbors the test accuracy is", knn.score(test_data[['PCR_08', 'PCR_10']].copy() ,test_data['risk'].copy()))

# Q5:
test_X = test_data[['PCR_08', 'PCR_10']].copy()
test_y = test_data['risk'].copy()

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(test_X, test_y)
visualize_clf(knn, test_X.to_numpy(), test_y.to_numpy(), 'Decision Reigons - k=1 neighbors', xlabel = 'PCR_08', ylabel = 'PCR_10')
print("with 1 neighbor the test accuracy is", knn.score(test_X ,test_y))


knn = KNeighborsClassifier(n_neighbors=18)
knn.fit(test_X, test_y)
visualize_clf(knn, test_X.to_numpy(), test_y.to_numpy(), 'Decision Reigons - k=18 neighbors', xlabel = 'PCR_08', ylabel = 'PCR_10')
print("with 18 neighbor the test accuracy is", knn.score(test_X ,test_y))

# Q6:
train_data = train_data[['risk',	'spread',	'covid', 'PCR_01', 'PCR_04', 'PCR_05', 'PCR_08', 'PCR_10', 'sport_activity', 'sugar_levels']]
y = train_data['risk']
X = train_data.drop(['risk',	'spread',	'covid'], axis = 1)
max = 0
k_opt = 0
k_range = list(range(1, 20, 1)) + list(range(20, 300, 10))
k_scores = []
k_train_scores = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_validate(knn, X, y, cv=8, return_train_score=True)
    mean = scores['test_score'].mean()
    mean_train = scores['train_score'].mean()
    if(mean > max):
      max = mean
      k_opt = k
    k_scores.append(mean)
    k_train_scores.append(mean_train)
plt.plot(k_range, k_scores, linewidth=2)
plt.plot(k_range, k_train_scores, linewidth=2)
plt.suptitle("Cross-Validated Accuracy vs. K value of knn")
plt.legend("Validation","Train")
plt.xlabel('Value of K for KNN')
plt.ylabel('Cross-Validated Accuracy')
plt.show()

print("the best k is: ", k_opt, "  and the mean is:", k_scores[k_opt])